In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [7]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth','PetalLength', 'PetalWidth','Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [8]:
train=pd.read_csv("iris_training.csv", names = CSV_COLUMN_NAMES, header=0)
test=pd.read_csv("iris_test.csv", names=CSV_COLUMN_NAMES, header=0)

In [9]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [10]:
def input_fn(features, labels, training=True, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [11]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None,

In [12]:
classifier = tf.estimator.DNNClassifier(feature_columns = my_feature_columns, hidden_units = [30,10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\btany\\AppData\\Local\\Temp\\tmp90g6csa6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
classifier.train(input_fn=lambda: input_fn(train,train_y, training=True), steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\btany\AppData\Local\Temp\tmp90g6csa6\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.2457014, step = 0
INFO:tensorflow:global_step/sec: 372.414
INFO:tensorflow:loss = 0.98833317, step = 100 (0.268 sec)
INFO:tensorflow:global_step/sec: 531.821
INFO:tensorflow:loss = 0.94550717, step = 200 (0.191 sec)


In [14]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('Test set accuracy: {accuracy:0.3f}'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-04-16T03:59:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\btany\AppData\Local\Temp\tmp90g6csa6\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.27149s
INFO:tensorflow:Finished evaluation at 2023-04-16-03:59:34
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.42273393, global_step = 5000, loss = 0.42273393
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\btany\AppData\Local\Temp\tmp90g6csa6\model.ckpt-5000
Test set accuracy: 0.967


In [16]:
batch_size=32
def input_fn(features, labels = 256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth','PetalLength', 'PetalWidth']
predict = {}

print('Please type numeric values as prompted')

for feature in features:
    valid = True
    while valid:
        val = input(feature + ':')
        if not val.isdigit(): valid=False
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100*probability))

Please type numeric values as prompted
SepalLength:0.2
SepalWidth:0.2
PetalLength:0.2
PetalWidth:0.2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\btany\AppData\Local\Temp\tmp90g6csa6\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Versicolor" (38.8%)
